In [ ]:
import graphviz
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score

# Load Data Sets

In [ ]:
driver_table = pd.read_csv("data/driver_ids.csv")
ride_table = pd.read_csv("data/ride_ids.csv")
ride_ts_table = pd.read_csv("data/ride_timestamps.csv")

In [ ]:
print("Minimum onboarding date: ", driver_table[['driver_onboard_date']].sort_values('driver_onboard_date').head(1)['driver_onboard_date']) 
print("Maximum onboarding date: ", driver_table[['driver_onboard_date']].sort_values('driver_onboard_date', ascending=False).head(1)['driver_onboard_date'])

In [ ]:
print("Minimum ride date: ", ride_ts_table[['timestamp']].sort_values('timestamp').head(1)['timestamp']) 
print("Maximum ride date: ", ride_ts_table[['timestamp']].sort_values('timestamp', ascending=False).head(1)['timestamp'])

In [ ]:
print("Number of drivers: ", len(np.unique(driver_table['driver_id'])))

In [ ]:
print("Number of rides: ", len(np.unique(ride_table['ride_id'])))

# Define Assumptions

In [ ]:
base_fare = 2
cost_per_mile = 1.15
cost_per_minute = 0.22
service_fee = 1.75
minimum_fare = 5
maximum_fare = 400
meters_to_miles = 1609.34
seconds_to_minutes = 60
#Lyft take 20% of a driver's earnings.
lyft_share = 0.2

# Calculate Earnings and normalize data units

In [ ]:
ride_table['ride_distance_miles'] = ride_table['ride_distance'] / meters_to_miles
ride_table['ride_duration_minute'] = ride_table['ride_duration'] / seconds_to_minutes
ride_table['ride_earnings'] = np.minimum(np.maximum(minimum_fare, (ride_table['ride_distance_miles'] * cost_per_mile + 
                                  ride_table['ride_duration_minute'] * cost_per_minute
                                  + base_fare + service_fee) * (ride_table['ride_prime_time']/100.0 + 1)), maximum_fare) * lyft_share
ride_table['unconstrained_ride_earnings'] = ride_table['ride_distance_miles'] * cost_per_mile + ride_table['ride_duration_minute'] * cost_per_minute + base_fare + service_fee

# Get Prime Time statistics

In [ ]:
x = (ride_table.groupby(['driver_id', 'ride_prime_time']).count() / ride_table.groupby(['driver_id']).count())[['ride_distance_miles']].reset_index().rename(columns = {'ride_distance_miles': 'percent_contribution_prime'})
prime = pd.pivot_table(x, index = 'driver_id', columns = 'ride_prime_time', values = 'percent_contribution_prime')
prime.fillna(0, inplace = True)
prime = prime.reset_index()
prime.columns = prime.columns.astype(str)
prime['prime_work'] = pd.DataFrame(prime[prime.columns.difference(['0'])].sum(axis = 1), columns = {'prime_percentage'})

In [ ]:
prime.mean()

# Days since onboarded

In [ ]:
ride_date = ride_ts_table.groupby('ride_id')['timestamp'].max()
ride_date = pd.DataFrame(ride_date).reset_index()
ride_table = pd.merge(ride_table, ride_date, on = 'ride_id')
ride_table = pd.merge(ride_table, driver_table, on = 'driver_id')
ride_table['days_since_onboarded'] = (pd.to_datetime(ride_table['timestamp']) - pd.to_datetime(ride_table['driver_onboard_date'])).astype('timedelta64[D]')

In [ ]:
ride_table['month_since_onboarded'] = np.floor((ride_table['days_since_onboarded'] )/ 31)
ride_table['biweek_since_onboarded'] = np.floor((ride_table['days_since_onboarded'] )/ 14)
ride_table['week_since_onboarded'] = np.floor((ride_table['days_since_onboarded'] )/ 7)
ride_table['max_time'] = (pd.to_datetime(ride_table['timestamp']).max() - pd.to_datetime(ride_table['driver_onboard_date'])).astype('timedelta64[D]') 

# Get the time between driver events

In [ ]:
ride_ts_table["time"] = pd.to_datetime(ride_ts_table["timestamp"],dayfirst=True)
group_ride_table = ride_ts_table.groupby('ride_id')['time']
ride_ts_table['time_diff'] = group_ride_table.diff()
ride_ts_table['time_seconds'] = ride_ts_table['time_diff'].fillna(0) / pd.datetools.timedelta(seconds=1)

# Days between rides

In [ ]:
ride_timings = pd.pivot_table(ride_ts_table, index = 'ride_id', columns = 'event', values = 'time_seconds')
result = ride_table.sort_values(['driver_id','days_since_onboarded'])\
           .groupby(['driver_id'])['days_since_onboarded']\
           .diff().fillna(0)
df = ride_table.sort_values(['driver_id','days_since_onboarded'])
df['days_diff_between_rides'] = result
print('95th percentile days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().quantile(0.95))
print('75th percentile days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().quantile(0.75))
print('50th percentile days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().quantile(0.5))
print('25th percentile days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().quantile(0.25))
print('Max days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().max())
print('Min days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().min())
print('Mean days diff between rides: ', df.groupby('driver_id')['days_diff_between_rides'].max().mean())

# Additional Features

In [ ]:
count_rides_1st_2weeks = ride_table[ride_table['biweek_since_onboarded'] == 0].groupby('driver_id')['ride_id'].count().reset_index()
count_prime_rides_1st_2weeks = ride_table[np.logical_and(ride_table['biweek_since_onboarded'] == 0, ride_table['ride_prime_time'] > 0)].groupby('driver_id')['ride_id'].count().reset_index()
total_earnings_1st_2weeks = ride_table[ride_table['biweek_since_onboarded'] == 0].groupby('driver_id')['ride_earnings'].sum().reset_index()
drivers = pd.DataFrame(np.unique(ride_table[ride_table['days_since_onboarded'] >= 14]['driver_id']))
drivers.rename(columns ={0:'driver_id'}, inplace=True)
count_rides_1st_2weeks.rename(columns={'ride_id': 'Number Rides first two weeks'}, inplace=True)
count_prime_rides_1st_2weeks.rename(columns={'ride_id': 'Number Prime Rides first two weeks'}, inplace=True)
total_earnings_1st_2weeks.rename(columns={'ride_earnings': 'Earnings first two weeks'}, inplace=True)
drivers_1 = pd.merge(drivers, count_rides_1st_2weeks, on='driver_id', how = 'inner')
drivers_2 = pd.merge(drivers_1, count_prime_rides_1st_2weeks, on='driver_id', how = 'inner')
drivers_3 = pd.merge(drivers_2, total_earnings_1st_2weeks, on='driver_id', how = 'inner')

# Prepare data for modelling

In [ ]:
final = df[np.logical_and(df['max_time'] >= 60, df['month_since_onboarded'] == 0 )]
final = pd.merge(final, ride_timings.reset_index(), on = 'ride_id')
final = pd.merge(final, prime.reset_index(), on='driver_id')
# final = pd.merge(final, drivers_3, on='driver_id', how = 'inner')
final.rename(columns={'accepted_at': 'time_between_request_and_accept', 'arrived_at': 'time_between_accept_arrive', 
            'picked_up_at': 'time_between_arrive_pickup', 'dropped_off_at': 'time_between_pickup_dropoff'}, inplace=True)
ride_table_3 = final[np.logical_and(final['max_time'] >= 60, final['month_since_onboarded'] == 0 )]

temp = ride_table_3.groupby(['driver_id']).mean().reset_index()[['driver_id', 'time_between_request_and_accept', 'time_between_accept_arrive', 
                                                                  'time_between_arrive_pickup', 'time_between_pickup_dropoff', 
                                                                    'ride_distance_miles', 'ride_duration_minute']]
x = pd.merge(temp, drivers_3, on = 'driver_id')

# Churn Definition

In [ ]:
churn_definition = pd.merge(pd.DataFrame(ride_table.groupby('driver_id')['days_since_onboarded'].max()).reset_index(), 
         pd.DataFrame(ride_table.groupby('driver_id')['max_time'].max()).reset_index(),
         on = 'driver_id')
churn_definition['churned'] = (churn_definition['max_time'] - churn_definition['days_since_onboarded']) > 31
dataset = pd.merge(x, churn_definition[['driver_id', 'churned']], on = 'driver_id', how = 'inner')
print(dataset['churned'].sum())

# Random Forest Churn Prediction

In [ ]:
np.random.seed(20)
final_data = dataset
scores =[]
rf = RandomForestClassifier() 
X = final_data[dataset.columns[2:len(dataset.columns)-1]]
y = final_data['churned']
msk = np.random.rand(len(X)) < 0.7
X_train = X[msk]
X_test = X[~msk]
y_train = y[msk]
y_test = y[~msk]
rf.fit(X_train, y_train)
y_score = rf.predict_proba(X_test)[:,1]
scores.append(roc_auc_score(y_test, y_score, average=None))
print("ROC/AUC score: ", scores[0])
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = final_data[dataset.columns[2:len(dataset.columns)-1]].columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

# Decision Tree Churn Prediction

In [ ]:
final_data = dataset
np.random.seed(2)

scores =[]
X = final_data[dataset.columns[2:len(dataset.columns)-1]]
y = final_data['churned']
msk = np.random.rand(len(X)) < 0.7
X_train = X[msk]
X_test = X[~msk]
y_train = y[msk]
y_test = y[~msk]

dtree=DecisionTreeClassifier(class_weight={0: 1, 1: 3}, criterion='gini', max_depth=5,
            max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=25, min_weight_fraction_leaf=0.0,
            presort=False, random_state=25, splitter='best')
dtree.fit(X_train, y_train) 
y_predict = dtree.predict_proba(X_test)[:,1]
print("ROC/AUC score: ",roc_auc_score(y_test, y_predict, average=None))
with open("churn.dot", "w") as f:
    f = tree.export_graphviz(dtree, out_file=f, feature_names = X_train.columns)
#Can see the results in webgraphviz

In [ ]:
dot_data = tree.export_graphviz(dtree, out_file=None, 
                         feature_names=X_train.columns,  
                         class_names=['not churn', 'churned'],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

# LTV Decision Tree

In [ ]:
#Wasn't working well - results too poor (not enough data to validate)

dataset_2 = pd.merge(x, ride_table.groupby(['driver_id'])['ride_earnings'].sum().reset_index(), on = 'driver_id')
np.random.seed(15)


final_data = dataset_2
scores =[]
dtree = tree
clf = dtree.DecisionTreeRegressor(criterion='friedman_mse', splitter='best', max_depth=10, min_samples_split=5, min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, max_features=None, random_state=40, 
                                  max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                  presort=False)
X = final_data[dataset_2.columns[1:len(dataset.columns)-2]]
y = final_data['ride_earnings']
msk = np.random.rand(len(X)) < 0.7
X_train = X[msk]
X_test = X[~msk]
y_train = y[msk]
y_test = y[~msk]
clf.fit(X_train, y_train)
y_score = clf.predict(X_test)
print(r2_score(y_score, y_test.reset_index()['ride_earnings']))
print(np.sqrt(mean_squared_error(y_score, y_test.reset_index()['ride_earnings'])))
with open("LTV.dot", "w") as f:
    f = tree.export_graphviz(clf, out_file=f, feature_names = X_train.columns)

# LTV Prediction RF


In [ ]:
#Wasn't working well - results too poor (not enough data to validate)

rf = RandomForestRegressor(random_state=40)
rf.fit(X_train, y_train)
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), X_train.columns), 
             reverse=True)

In [ ]:
y_score = rf.predict(X_test)
print(r2_score(y_score, y_test.reset_index()['ride_earnings']))
print(np.sqrt(mean_squared_error(y_score, y_test.reset_index()['ride_earnings'])))


# Cohorts

In [ ]:
plt.figure(figsize=(30,10))
plt.hist(ride_table['ride_earnings'], bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram of Lyft Driver earnings per ride")
plt.show()

In [ ]:
#Mean monthly earnings for people for who we have most data.
ride_table_2 = ride_table[ride_table['max_time'] >= 90]
cohort_analysis = ride_table_2.groupby(['driver_id', 'month_since_onboarded'])['ride_earnings'].sum().reset_index()
cohort_analysis.groupby('month_since_onboarded')['ride_earnings'].mean()

In [ ]:
print("Standard LTV calculation: ",  230 / 0.15)

# Cohort definitions

In [ ]:
import datetime
print(driver_table['driver_onboard_date'].max(), driver_table['driver_onboard_date'].min())
pd.to_datetime(driver_table['driver_onboard_date'].max()) - pd.to_datetime(driver_table['driver_onboard_date'].min())
print(pd.to_datetime(driver_table['driver_onboard_date'].min()) + datetime.timedelta(days=12))
print(pd.to_datetime(driver_table['driver_onboard_date'].min()) + datetime.timedelta(days=24))
print(pd.to_datetime(driver_table['driver_onboard_date'].min()) + datetime.timedelta(days=36))
print(pd.to_datetime(driver_table['driver_onboard_date'].min()) + datetime.timedelta(days=48))


In [ ]:
#Bi-weekly cohorts
cohort = [None,None,None,None]
cohort[0] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-03-28', ride_table['driver_onboard_date'] < '2016-04-09')]
cohort[1] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-09', ride_table['driver_onboard_date'] < '2016-04-21')]
cohort[2] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-21', ride_table['driver_onboard_date'] < '2016-05-03')]
cohort[3] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-05-03', ride_table['driver_onboard_date'] <= '2016-05-15')]

In [ ]:
#Monthly cohorts

# cohort = [None,None]

# cohort[0] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-03-28', ride_table['driver_onboard_date'] < '2016-04-21')]
# cohort[1] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-21', ride_table['driver_onboard_date'] < '2016-05-15')]


In [ ]:
active = []
new = []
transactions = []
money = []
for i in range(0, len(cohort)):
    active.append([])
    new.append([])
    transactions.append([])
    money.append([])
    for j in range(0, 6):
        new[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['biweek_since_onboarded'] == 0])))
        if j >= 4 and cohort[i]['max_time'].quantile(0.01) < 80:
            continue
        elif j >= 2 and cohort[i]['max_time'].quantile(0.01) < 50:
            continue
        active[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['biweek_since_onboarded'] == j])))
        transactions[i].append(len(np.unique(cohort[i]['ride_id'][cohort[i]['biweek_since_onboarded'] == j])))
        money[i].append(np.sum(cohort[i]['ride_earnings'][cohort[i]['biweek_since_onboarded'] <= j]))
cohort_active_users = pd.DataFrame(active)
cohort_active_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'biweek 0',
                                     1: 'biweek 1',
                                     2: 'biweek 2',
                                    3: 'biweek 3',
                                    4: 'biweek 4',
                                    5: 'biweek 5',
                                    6: 'biweek 6'}
                           , inplace=True)
cohort_new_users = pd.DataFrame(new)
cohort_new_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'biweek 0',
                                     1: 'biweek 1',
                                     2: 'biweek 2',
                                    3: 'biweek 3',
                                    4: 'biweek 4',
                                    5: 'biweek 5',
                                    6: 'biweek 6'}
                           , inplace=True)
cohort_transactions = pd.DataFrame(transactions)
cohort_transactions.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'biweek 0',
                                     1: 'biweek 1',
                                     2: 'biweek 2',
                                    3: 'biweek 3',
                                    4: 'biweek 4',
                                    5: 'biweek 5',
                                    6: 'biweek 6'}
                           , inplace=True)
cohort_money = pd.DataFrame(money)
cohort_money.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'biweek 0',
                                     1: 'biweek 1',
                                     2: 'biweek 2',
                                    3: 'biweek 3',
                                    4: 'biweek 4',
                                    5: 'biweek 5',
                                    6: 'biweek 6'}
                           , inplace=True)
CustomerRetention = cohort_active_users / cohort_new_users
TransactionsPerCustomer = cohort_transactions/cohort_active_users
AmountPerTransaction = cohort_money/cohort_transactions
HistoricCLV=cohort_money/cohort_new_users
print(CustomerRetention)
print(TransactionsPerCustomer)
print(AmountPerTransaction)
print(HistoricCLV)
print(cohort_active_users)
LTV = pd.DataFrame((HistoricCLV * cohort_active_users).sum()) / pd.DataFrame(cohort_active_users.sum())

LTV.rename(columns ={0: 'HistoricLTV'}, inplace='True')
LTV['Percent Increase'] = LTV.diff() / LTV.shift(1)
print(LTV)

In [ ]:
plt.plot(LTV['HistoricLTV'])

In [ ]:
plt.title('Percent increase in LTV every 2 weeks')
plt.xticks([0,1,2,3,4,5], ['2 weeks', '4 weeks', '6 weeks', '8 weeks', '10 weeks', '12 weeks'], rotation='vertical')
plt.plot([0,1,2,3,4,5], LTV['Percent Increase'] * 100)

# Fit Model to predict LTV

In [ ]:
regr = linear_model.LinearRegression(fit_intercept=True)
x = [[1],[2],[3],[4], [5]]
# Train the model using the training sets
regr.fit(x, np.log(LTV['Percent Increase'])[1:].reset_index()['Percent Increase'].reshape(-1,1))

# Make predictions using the testing set
# diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_, regr.intercept_)
x_test = []
for i in range(1, 100):
    x_test.append([i])
multiple = reduce(lambda x, y: x*y, np.exp(regr.predict(x_test)) + 1)
multiple_2 = reduce(lambda x, y: x*y, np.exp(regr.predict(x_test))[5:] + 1)
print("LTV: ", multiple_2 * 633.483576)
#multiple * 141.583565

# Monthly cohort tracking (Should be more accurate measure of churn but more limited data)

In [ ]:
active = []
new = []
transactions = []
money = []
for i in range(0, len(cohort)):
    active.append([])
    new.append([])
    transactions.append([])
    money.append([])
    for j in range(0, 3):
        new[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['month_since_onboarded'] == 0])))
        if j == 2 and cohort[i]['max_time'].quantile(0.1) < 80:
            continue
        elif j == 1 and cohort[i]['max_time'].quantile(0.1) < 50:
            continue
        active[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['month_since_onboarded'] == j])))
        transactions[i].append(len(np.unique(cohort[i]['ride_id'][cohort[i]['month_since_onboarded'] == j])))
        money[i].append(np.sum(cohort[i]['ride_earnings'][cohort[i]['month_since_onboarded'] <= j]))
cohort_active_users = pd.DataFrame(active)
cohort_active_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'Month 0',
                                     1: 'Month 1',
                                     2: 'Month 2'}
                           , inplace=True)
cohort_new_users = pd.DataFrame(new)
cohort_new_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'Month 0',
                                     1: 'Month 1',
                                     2: 'Month 2'}
                           , inplace=True)
cohort_transactions = pd.DataFrame(transactions)
cohort_transactions.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'Month 0',
                                     1: 'Month 1',
                                     2: 'Month 2'}
                           , inplace=True)
cohort_money = pd.DataFrame(money)
cohort_money.rename(index={0:'Cohort 03-28 - 04-09', 
                                   1: 'Cohort 04-09 - 04-21', 
                                   2: 'Cohort 04-21 - 05-03', 
                                   3: 'Cohort 05-03 - 05-15'},
                           columns= {0: 'Month 0',
                                     1: 'Month 1',
                                     2: 'Month 2'}
                           , inplace=True)
CustomerRetention = cohort_active_users / cohort_new_users
TransactionsPerCustomer = cohort_transactions/cohort_active_users
AmountPerTransaction = cohort_money/cohort_transactions
HistoricCLV=cohort_money/cohort_new_users
print(CustomerRetention)
print(TransactionsPerCustomer)
print(AmountPerTransaction)
print(HistoricCLV)
LTV = pd.DataFrame((HistoricCLV * cohort_active_users).sum()) / pd.DataFrame(cohort_active_users.sum())

LTV.rename(columns ={0: 'HistoricLTV'}, inplace='True')
LTV['Percent Increase'] = LTV.diff() / LTV.shift(1)
LTV

In [ ]:
regr = linear_model.LinearRegression(fit_intercept=True)
x = [[1], [2]]
# Train the model using the training sets
regr.fit(x, np.log(LTV['Percent Increase'])[1:3].reset_index()['Percent Increase'].reshape(-1,1))
# regr.fit(x, LTV['Percent Increase'][1:3].reshape(-1,1))

# Make predictions using the testing set
# diabetes_y_pred = regr.predict(diabetes_X_test)
plt.plot([1,2], np.log(LTV['Percent Increase'])[1:3])
# The coefficients
print('Coefficients: \n', regr.coef_, regr.intercept_)
x_test = []
for i in range(1, 100):
    x_test.append([i])
multiple = reduce(lambda x, y: x*y, np.exp(regr.predict(x_test)) + 1)
multiple_2 = reduce(lambda x, y: x*y, np.exp(regr.predict(x_test))[2:] + 1)
print("LTV from monthly view: " , multiple_2 * 652) 
#multiple * 299

In [ ]:
248 * 0.85 * 0.85,248 * 0.85 

In [ ]:
print("Life Time of driver: ", 1 / 0.15)

In [ ]:
#Not working because probably not enough data.

# from lifelines import KaplanMeierFitter
# kmf = KaplanMeierFitter() 
# cohort = [None,None,None,None]
# cohort[0] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-03-28', ride_table['driver_onboard_date'] < '2016-04-09')]
# cohort[1] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-09', ride_table['driver_onboard_date'] < '2016-04-21')]
# cohort[2] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-21', ride_table['driver_onboard_date'] < '2016-05-03')]
# cohort[3] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-05-03', ride_table['driver_onboard_date'] <= '2016-05-15')]
# churn_definition = pd.merge(pd.DataFrame(cohort[0].groupby('driver_id')['days_since_onboarded', 'month_since_onboarded'].max()).reset_index(), 
#          pd.DataFrame(cohort[0].groupby('driver_id')['max_time'].max()).reset_index(),
#          on = 'driver_id')
# churn_definition['churned'] = (churn_definition['max_time'] - churn_definition['days_since_onboarded']) > 23
# # The 1st arg accepts an array or pd.Series of individual survival times
# # The 2nd arg accepts an array or pd.Series that indicates if the event 
# # interest (or death) occured.
# final = churn_definition

# kmf.fit(durations = final['month_since_onboarded'], 
#         event_observed =final['churned'])
# kmf.event_table


In [ ]:
from scipy.stats import expon, exponweib
rv = exponweib(4, 2)
rv.pdf(0)

In [ ]:
from scipy.stats import expon, exponweib
rv = exponweib(4, 0.9)
rv.pdf(3)

In [ ]:
from scipy.stats import expon, exponweib
rv = exponweib(4, 0.9)
x = range(3, 20)

m = reduce(lambda x, y: x*y, rv.pdf(x) + 1)
m * 652

In [ ]:
from scipy.stats import expon, exponweib
rv = exponweib(4, 2)
x = range(3, 20)

m = reduce(lambda x, y: x*y, rv.pdf(x) + 1)

# Segmentation

In [ ]:
cohort = [None,None,None,None]
cohort[0] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-03-28', ride_table['driver_onboard_date'] < '2016-04-09')]
cohort[1] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-09', ride_table['driver_onboard_date'] < '2016-04-21')]
cohort[2] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-04-21', ride_table['driver_onboard_date'] < '2016-05-03')]
cohort[3] = ride_table[np.logical_and(ride_table['driver_onboard_date'] >= '2016-05-03', ride_table['driver_onboard_date'] <= '2016-05-15')]

In [ ]:
count_rides_1st_2weeks = ride_table[ride_table['biweek_since_onboarded'] == 0].groupby('driver_id')['ride_id'].count().reset_index()
count_prime_rides_1st_2weeks = ride_table[np.logical_and(ride_table['biweek_since_onboarded'] == 0, ride_table['ride_prime_time'] > 0)].groupby('driver_id')['ride_id'].count().reset_index()
total_earnings_1st_2weeks = ride_table[ride_table['biweek_since_onboarded'] == 0].groupby('driver_id')['ride_earnings'].sum().reset_index()
drivers = pd.DataFrame(np.unique(ride_table[ride_table['days_since_onboarded'] >= 14]['driver_id']))
drivers.rename(columns ={0:'driver_id'}, inplace=True)
count_rides_1st_2weeks.rename(columns={'ride_id': 'Number Rides first two weeks'}, inplace=True)
count_prime_rides_1st_2weeks.rename(columns={'ride_id': 'Number Prime Rides first two weeks'}, inplace=True)
total_earnings_1st_2weeks.rename(columns={'ride_earnings': 'Earnings first two weeks'}, inplace=True)
drivers_1 = pd.merge(drivers, count_rides_1st_2weeks, on='driver_id', how = 'inner')
drivers_2 = pd.merge(drivers_1, count_prime_rides_1st_2weeks, on='driver_id', how = 'inner')
drivers_3 = pd.merge(drivers_2, total_earnings_1st_2weeks, on='driver_id', how = 'inner')

In [ ]:
count_rides_1st_2weeks['1st2weeksDistance'] = count_rides_1st_2weeks['Number Rides first two weeks'] > 42.5
count_prime_rides_1st_2weeks['not_enough_prime'] = count_prime_rides_1st_2weeks['Number Prime Rides first two weeks'] <= 5
ride_timings['too_long_topickup'] = ride_timings['arrived_at'] <= 369.5

In [ ]:
def cohort_analysis(num_rides_1st_2weeks, cohort):
    cohort[0] = num_rides_1st_2weeks[np.logical_and(num_rides_1st_2weeks['driver_onboard_date'] >= '2016-03-28', num_rides_1st_2weeks['driver_onboard_date'] < '2016-04-09')]
    cohort[1] = num_rides_1st_2weeks[np.logical_and(num_rides_1st_2weeks['driver_onboard_date'] >= '2016-04-09', num_rides_1st_2weeks['driver_onboard_date'] < '2016-04-21')]
    cohort[2] = num_rides_1st_2weeks[np.logical_and(num_rides_1st_2weeks['driver_onboard_date'] >= '2016-04-21', num_rides_1st_2weeks['driver_onboard_date'] < '2016-05-03')]
    cohort[3] = num_rides_1st_2weeks[np.logical_and(num_rides_1st_2weeks['driver_onboard_date'] >= '2016-05-03', num_rides_1st_2weeks['driver_onboard_date'] <= '2016-05-15')]
    active = []
    new = []
    transactions = []
    money = []
    for i in range(0, len(cohort)):
        active.append([])
        new.append([])
        transactions.append([])
        money.append([])
        for j in range(0, 3):
            new[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['month_since_onboarded'] == 0])))
            if j == 2 and cohort[i]['max_time'].quantile(0.1) < 80:
                continue
            elif j == 1 and cohort[i]['max_time'].quantile(0.1) < 50:
                continue
            active[i].append(len(np.unique(cohort[i]['driver_id'][cohort[i]['month_since_onboarded'] == j])))
            transactions[i].append(len(np.unique(cohort[i]['ride_id'][cohort[i]['month_since_onboarded'] == j])))
            money[i].append(np.sum(cohort[i]['ride_earnings'][cohort[i]['month_since_onboarded'] <= j]))
    cohort_active_users = pd.DataFrame(active)
    cohort_active_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                       1: 'Cohort 04-09 - 04-21', 
                                       2: 'Cohort 04-21 - 05-03', 
                                       3: 'Cohort 05-03 - 05-15'},
                               columns= {0: 'Month 0',
                                         1: 'Month 1',
                                         2: 'Month 2'}
                               , inplace=True)
    cohort_new_users = pd.DataFrame(new)
    cohort_new_users.rename(index={0:'Cohort 03-28 - 04-09', 
                                       1: 'Cohort 04-09 - 04-21', 
                                       2: 'Cohort 04-21 - 05-03', 
                                       3: 'Cohort 05-03 - 05-15'},
                               columns= {0: 'Month 0',
                                         1: 'Month 1',
                                         2: 'Month 2'}
                               , inplace=True)
    cohort_transactions = pd.DataFrame(transactions)
    cohort_transactions.rename(index={0:'Cohort 03-28 - 04-09', 
                                       1: 'Cohort 04-09 - 04-21', 
                                       2: 'Cohort 04-21 - 05-03', 
                                       3: 'Cohort 05-03 - 05-15'},
                               columns= {0: 'Month 0',
                                         1: 'Month 1',
                                         2: 'Month 2'}
                               , inplace=True)
    cohort_money = pd.DataFrame(money)
    cohort_money.rename(index={0:'Cohort 03-28 - 04-09', 
                                       1: 'Cohort 04-09 - 04-21', 
                                       2: 'Cohort 04-21 - 05-03', 
                                       3: 'Cohort 05-03 - 05-15'},
                               columns= {0: 'Month 0',
                                         1: 'Month 1',
                                         2: 'Month 2'}
                               , inplace=True)
    CustomerRetention = cohort_active_users / cohort_new_users
    TransactionsPerCustomer = cohort_transactions/cohort_active_users
    AmountPerTransaction = cohort_money/cohort_transactions
    HistoricCLV=cohort_money/cohort_new_users
    print(CustomerRetention)
    print(TransactionsPerCustomer)
    print(AmountPerTransaction)
    print(HistoricCLV)
    LTV = pd.DataFrame((HistoricCLV * cohort_active_users).sum()) / pd.DataFrame(cohort_active_users.sum())
    print(cohort_active_users)
    LTV.rename(columns ={0: 'HistoricLTV'}, inplace='True')
    LTV['Percent Increase'] = LTV.diff() / LTV.shift(1)
    print(LTV)

# More than > 42.5 rides in the first 2 weeks

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, count_rides_1st_2weeks[count_rides_1st_2weeks['1st2weeksDistance'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Less than 42.5 rides in the first 2 weeks

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, count_rides_1st_2weeks[count_rides_1st_2weeks['1st2weeksDistance'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Less than 5 prime rides in the first 2 weeks

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, count_prime_rides_1st_2weeks[count_prime_rides_1st_2weeks['not_enough_prime'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# More than 5 prime time rides in the first 2 weeks.

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, count_prime_rides_1st_2weeks[count_prime_rides_1st_2weeks['not_enough_prime'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Greater than 1 day between rides on average

In [ ]:
df_1 = df.groupby('driver_id')['days_diff_between_rides'].mean().reset_index()
df_1['not_very_active'] = df_1['days_diff_between_rides'] <= 1
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, df_1[df_1['not_very_active'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Less than one day on average between rides

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, df_1[df_1['not_very_active'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# More than 3 minutes on average between accepted ride and arrived to pick up

In [ ]:
ride_timings_2 = pd.merge(ride_timings.reset_index(), ride_table, on = 'ride_id')
ride_timings_2 = ride_timings_2.groupby('driver_id')['arrived_at'].median().reset_index()
ride_timings_2['too_long_topickup'] = ride_timings_2['arrived_at'] <= 180
ride_timings_2['too_long_topickup'].sum(), ride_timings_2['too_long_topickup'].count()
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_topickup'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Less than 3 minutes on average between accepted ride and arrived to pick up


In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_topickup'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Time between request and accept greater than 5 seconds

In [ ]:
ride_timings_2 = pd.merge(ride_timings.reset_index(), ride_table, on = 'ride_id')
ride_timings_2 = ride_timings_2.groupby('driver_id')['accepted_at'].median().reset_index()
ride_timings_2['too_long_wait'] = ride_timings_2['accepted_at'] <= 5
ride_timings_2['too_long_wait'].sum(), ride_timings_2['too_long_wait'].count()
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_wait'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Time between request and accept less than 5 seconds

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_wait'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Time between arriving and picking-up passenger less than 2.4 seconds

In [ ]:
ride_timings_2 = pd.merge(ride_timings.reset_index(), ride_table, on = 'ride_id')
ride_timings_2 = ride_timings_2.groupby('driver_id')['picked_up_at'].mean().reset_index()
ride_timings_2['too_long_wait'] = ride_timings_2['picked_up_at'] >= 2.4
ride_timings_2['too_long_wait'].sum(), ride_timings_2['too_long_wait'].count()
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_wait'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Time between arriving and picking-up passenger greater than 2.4 seconds

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_timings_2[ride_timings_2['too_long_wait'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Ride Duration in Minutes greater than 13 minutes

In [ ]:
ride_table_2 = ride_table.groupby('driver_id')['ride_duration_minute'].mean().reset_index()
ride_table_2['too_short_ride'] = ride_table_2['ride_duration_minute'] < 13
ride_table_2['too_short_ride'].sum(), ride_table_2['too_short_ride'].count()
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_table_2[ride_table_2['too_short_ride'] == 0], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Ride Duration in Minutes less than 13 minutes

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, ride_table_2[ride_table_2['too_short_ride'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)

# Recommendation

In [ ]:
cohort = [None,None,None,None]
num_rides_1st_2weeks = pd.merge(ride_table, count_prime_rides_1st_2weeks[count_prime_rides_1st_2weeks['not_enough_prime'] == 0], on = 'driver_id', how = 'inner') 
num_rides_1st_2weeks = pd.merge(num_rides_1st_2weeks, count_rides_1st_2weeks[count_rides_1st_2weeks['1st2weeksDistance'] == 1], on = 'driver_id', how = 'inner') 
cohort_analysis(num_rides_1st_2weeks, cohort)